# Forward and Inverse Kinematics Tests

In [1]:
from sympy import symbols, cos, sin, asin, acos, pi, sqrt, atan2, Abs, simplify, pprint, solveset, S
from sympy.matrices import Matrix, Transpose
import numpy as np

In [2]:
dtr = pi/180.0
rtd = 180.0/pi

# init_printing()

## Forward Kinematics

In [3]:

# Define symbols
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')


# DH Parameters
s = {alpha0: 0,     a0: 0,      d1: 0.75,  q1: q1,           # 1
     alpha1: -pi/2, a1: 0.35,   d2: 0,     q2: -pi/2 + q2,   # 2
     alpha2: 0,     a2: 1.25,   d3: 0,     q3: q3,           # 3
     alpha3: -pi/2, a3: -0.054, d4: 1.5,   q4: q4,           # 4
     alpha4: pi/2,  a4: 0,      d5: 0,     q5: q5,           # 5
     alpha5: -pi/2, a5: 0,      d6: 0,     q6: q6,           # 6
     alpha6: 0,     a6: 0,      d7: 0.303, q7: 0}            # G

# Rotation Functions
def rot_x(q):
    T = Matrix([[1, 0, 0, 0],
                [0, cos(q), -sin(q), 0],
                [0, sin(q), cos(q), 0],
                [0, 0, 0, 1]])
    return T

def trans_x(d):
    T = Matrix([[1, 0, 0, d],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T

def rot_z(q):
    T = Matrix([[cos(q), -sin(q), 0, 0],
                [sin(q), cos(q), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
    return T


def trans_z(d):
    T = Matrix([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, d],
                [0, 0, 0, 1]])
    return T

def rot_y(q):
    T = Matrix([[cos(q),  0, sin(q), 0],
                [0,       1,      0, 0],
                [-sin(q), 0, cos(q), 0],
                [0,       0,      0, 1]])
    return T



def dh_trans(alpha0, a0, d1, q1):
    T = rot_x(alpha0) * trans_x(a0) * rot_z(q1) * trans_z(d1)
    return T

T0_1 = dh_trans(alpha0, a0, d1, q1)
T0_1 = T0_1.subs(s)

T1_2 = dh_trans(alpha1, a1, d2, q2)
T1_2 = T1_2.subs(s)

T2_3 = dh_trans(alpha2, a2, d3, q3)
T2_3 = T2_3.subs(s)

T3_4 = dh_trans(alpha3, a3, d4, q4)
T3_4 = T3_4.subs(s)

T4_5 = dh_trans(alpha4, a4, d5, q5)
T4_5 = T4_5.subs(s)

T5_6 = dh_trans(alpha5, a5, d6, q6)
T5_6 = T5_6.subs(s)

T6_G = dh_trans(alpha6, a6, d7, q7)
T6_G = T6_G.subs(s)


# Combine Transforms

T0_2 = T0_1 * T1_2
T0_3 = T0_2 * T2_3
T0_4 = T0_3 * T3_4
T0_5 = T0_4 * T4_5
T0_6 = T0_5 * T5_6
T0_G = T0_6 * T6_G


# Corrections for URDF/Gazebo reference frames
R2_corr = rot_z(pi/2) * rot_x(pi/2)
R3_corr = R2_corr
R4_corr = rot_z(pi) * rot_y(-pi/2)
R5_corr = R2_corr
R6_corr = R4_corr
RG_corr = R4_corr

# Apply corrections (so we could compare with ROS output)
T0_2_corr = T0_2 * R2_corr
T0_3_corr = T0_3 * R3_corr
T0_4_corr = T0_4 * R4_corr
T0_5_corr = T0_5 * R5_corr
T0_6_corr = T0_6 * R6_corr
T0_G_corr = T0_G * RG_corr

# Define helper functions
def forward_kinematics(precision = 6, subs = {}):
    return T0_G.evalf(precision, subs=subs)

def grip_corr(m):
    r = m * RG_corr
    return r.evalf()



# T1 = dh_trans(45*dtr, 5, 2, 30*dtr)
# T2 = rot_x(45*dtr) * trans_x(5) * rot_z(30*dtr) * trans_z(2)

# print('T0_1 =')
# pprint(T0_1)
# print('T1_2 =')
# pprint(T1_2)
# print('T0_2 =')
# pprint(T0_2)

# Debug outputs

'''
print('T0_1 =')
# pprint(T0_1)
print('T0_1 =')
pprint(T0_1.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T1_2 =')
# pprint(T1_2)
print('T1_2 =')
pprint(T1_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_2 =')
# pprint(T0_2)
print('T0_2 =')
pprint(T0_2.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T3_4 =')
# pprint(T3_4)
print('T3_4 =')
pprint(T3_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4 =')
# pprint(T0_4)
print('T0_4 =')
pprint(T0_4.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T4_5 =')
# pprint(T4_5)
print('T4_5 =')
pprint(T4_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5 =')
# pprint(T0_5)
print('T0_5 =')
pprint(T0_5.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T5_6 =')
# pprint(T5_6)
print('T5_6 =')
pprint(T5_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6 =')
# pprint(T0_6)
print('T0_6 =')
pprint(T0_6.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T6_G =')
# pprint(T6_G)
print('T6_G =')
pprint(T6_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_G =')
# pprint(T0_G)
print('T0_G =')
pprint(T0_G.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))






print('T0_2_corr =')
# pprint(T0_2_corr)
print('T0_2_corr =')
pprint(T0_2_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_3_corr =')
# pprint(T0_3_corr)
print('T0_3_corr =')
pprint(T0_3_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_4_corr =')
# pprint(T0_4_corr)
print('T0_4_corr =')
pprint(T0_4_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


print('T0_5_corr =')
# pprint(T0_5_corr)
print('T0_5_corr =')
pprint(T0_5_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))

print('T0_6_corr =')
# pprint(T0_6_corr)
print('T0_6_corr =')
pprint(T0_6_corr.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))


'''



# print('T0_G_corr =')
# pprint(T0_G_corr)
# print('T0_G_corr_simp =')
# s = simplify(T0_G_corr)
# pprint(s)
print('T0_G_corr =')
# pprint(s.evalf(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}))
pprint(grip_corr(forward_kinematics(subs={q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0})))




T0_G_corr =
⎡1.0   0    0   2.15299987792969⎤
⎢                               ⎥
⎢ 0   1.0   0          0        ⎥
⎢                               ⎥
⎢ 0    0   1.0  1.94600009918213⎥
⎢                               ⎥
⎣ 0    0    0         1.0       ⎦


## Test Forward Kinematics

In [4]:
# Test Forward Kinematics

# NOTE: Play with angles to test forward and inverse kinematics
joint_1 = 0.1
joint_2 = 0.3
joint_3 = 0.1
joint_4 = -0.15
joint_5 = 0.2 # 0, pi - corner cases
joint_6 = 0.1


print('O = T0_G =')
# O = T0_G.evalf(6, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6})
O = grip_corr(forward_kinematics(subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6}))
pprint(O.evalf(6))


# print('T0_G_corr =')
# pprint(T0_G_corr.evalf(6, subs={q1: joint_1, q2: joint_2, q3: joint_3, q4: joint_4, q5: joint_5, q6: joint_6}))




O = T0_G =
⎡0.825041    -0.10163   0.555859  2.31956 ⎤
⎢                                         ⎥
⎢0.0529424   0.993269   0.103023  0.223691⎥
⎢                                         ⎥
⎢-0.562588  -0.0555699  0.824868  1.13984 ⎥
⎢                                         ⎥
⎣    0          0          0        1.0   ⎦


## Inverse Kinematics Pipeline

In [7]:
# Inverse Kinematics Pipeline


# Orientation of EE
# roll = -0.543361935706796
# pitch = 1.02689551597329
# yaw = 1.35363377393308

# Return to URDF world
O1 = O * Transpose(RG_corr)

print('O1 =')
pprint(O1.evalf(6))

def get_rpy(m):
    # Source: http://planning.cs.uiuc.edu/node103.html
    alpha = atan2(m[1,0], m[0,0])
    beta = atan2(-m[2,0], sqrt(m[2,1] + m[2,2]))
    gamma = atan2(m[2,1], m[2,2])
    return gamma, beta, alpha

def get_rot_mat(r, p, y):
    Rxyz =  rot_z(y) * rot_y(p) * rot_x(r)
    return Rxyz


def inverse_kinematics(O):
    
    # Position of EE that we should reach
    px = O[0,3] 
    py = O[1,3] 
    pz = O[2,3] 

    # Get final rotation matrix
    R = O[0:3, 0:3]  # get_rot_mat(roll, pitch, yaw)
    # print('R =')
    # pprint(R)

    # Distance from wrist center to end-effector
    d = s[d7]

    # Wrist Center
    xc = px - d * R[0, 2]
    yc = py - d * R[1, 2]
    zc = pz - d * R[2, 2]

    # print('xc =', xc)
    # print('yc =', yc)
    # print('zc =', zc)

    # theta_1
    theta_1 = atan2(yc, xc)
    # print('theta_1 =', theta_1.evalf())

    # joint_2 center
    x0 = s[a1]
    z0 = s[d1]
    # print('x0 =', x0, ', z0 =', z0)

    # Link
    l1 = s[a2]
    # print('s[a3] = ', s[a3], ', s[d4] =', s[d4])
    l2 = sqrt(s[a3]**2 + s[d4]**2)
    theta30 = atan2(Abs(s[a3]), s[d4])
    # print('l1 =', l1, ', l2 =', l2)
    # print('theta30 =', theta30)

    # Dist from {0} to {wc}
    dwc = sqrt((sqrt(xc**2+yc**2)-x0)**2 + (zc-z0)**2)
    # print('dwc =', dwc)

    # Check validity
    assert(l1 + l2 > dwc)

    # Cosine law for cos(pi - beta):
    D = (l1**2 + l2**2 - dwc**2) / (2 * l1 * l2)
    # theta_3 = pi/2 - theta30 - atan2(sqrt(1 - D**2), D) # here is possible a second solution when elbow-down with "-" sign
    # theta_3_alt = pi/2 - theta30 - atan2(-sqrt(1 - D**2), D) # alt solution
    theta_3 = pi/2 - theta30 - acos(D) # here is possible a second solution when elbow-down with "-" sign
    theta_3_alt = pi/2 - theta30 + acos(D) # alt solution
    # print('D =', D)
    # print('theta_3 =', theta_3.evalf())
    # print('theta_3_alt =', theta_3_alt.evalf())

    # angle to l2 from l1
    bet = pi/2 + theta_3 + theta30
    # print('bet =', bet)

    # Find angle to {0} - {wc} line
    gam = atan2(sqrt(xc**2+yc**2) - x0, zc - z0)
    # print('gam =', gam)

    # Find angle between {0} - {wc} /_ l1
    # cos (alph) = E
    E = (l1**2 + dwc**2 - l2**2) / (2 * l1 * dwc)
    alph = acos(E)
    # print('E =', E)
    # print('alph =', alph)

    # if 0 < beta < pi then theta_2 = gam - alph else theta_2 = gam + alph
    if 0 < bet and bet < pi:
        theta_2 = gam - alph
    else:
        theta_2 = gam + alph
    # print('theta_2 =', theta_2)

    # pitch2 = theta_2 + theta_3
    # print('pitch2 =', pitch2.evalf())

    # Check by feeding angles to forward kinematics
    t04 = T0_4.evalf(subs={q1: theta_1, q2: theta_2, q3: theta_3, q4: 0})
    # print('wc_calc =')
    # pprint(t04)

    # Wrist center should be equal to what we've started from
    # print('xc_diff =', xc - t04[0,3])
    # print('yc_diff =', yc - t04[1,3])
    # print('zc_diff =', zc - t04[2,3])
    assert(abs(xc - t04[0,3]) < 1e-6)
    assert(abs(yc - t04[1,3]) < 1e-6)
    assert(abs(zc - t04[2,3]) < 1e-6)

    # Get R0_4 with theta_4 = 0
    R0_4 = t04[0:3, 0:3]
    # print('R0_4 =')
    # pprint(R0_4.evalf(6))

    # Calc R4_G - wrist rotation matrix
    R4_G = Transpose(R0_4) * R[0:3, 0:3]
    # print('R4_G =')
    # pprint(R4_G.evalf(6))

    # Calc symbolicaly wrist rotation matrix
    R4_G_sym = rot_z(q4) * rot_x(pi/2) * rot_z(q5) * rot_x(-pi/2) * rot_z(q6)

    '''
    # Show formula (for derivation)
    print('R4_G_sym =')
    pprint(simplify(R4_G_sym)) # just for visualization

    # Corner case q5 = 0
    print('R4_G_sym (q5=0) =')
    pprint(simplify(R4_G_sym.subs({q5: 0}))) # just for visualization

    # Corner case q5 = pi
    print('R4_G_sym (q5=pi) =')
    pprint(simplify(R4_G_sym.subs({q5: pi}))) # just for visualization
    '''


    # Solve wrist rotation matrix
    
    # inspired by https://www.geometrictools.com/Documentation/EulerAngles.pdf
    if R4_G[2,2] < 1:

        if R4_G[2,2] > -1:
            # theta_5 between (0, pi)
            theta_5 = acos(R4_G[2,2])
            theta_4 = atan2(-R4_G[1,2], -R4_G[0,2])
            theta_6 = atan2(-R4_G[2,1], R4_G[2,0])

        else:
            # q5 = pi s.t. cos(q5) = -1 and sin(q5) = 0
            theta_5 = pi
            # Not a unique solution: q4 - q6 = atan2(-r10,-r00)
            theta_4 = S(0)
            theta_6 = - atan2(-R4_G[1,0], -R4_G[0,0])

    else:
        # q5 = 0 s.t. cos(q5) = 1 and sin(q5) = 0 
        theta_5 = S(0)
        # Not a unique solution: q4 + q6 = atan2(r10, r00)
        theta_4 = S(0)
        theta_6 = atan2(R4_G[0,1], R4_G[0,0])




    # print('theta_4 =', theta_4)
    # print('theta_5 =', theta_5)
    # print('theta_6 =', theta_6)
    return theta_1, theta_2, theta_3, theta_4, theta_5, theta_6




theta_1, theta_2, theta_3, theta_4, theta_5, theta_6 = inverse_kinematics(O1)


# R4_G_check = R4_G_sym.evalf(6, subs = {q4: theta_4, q5: theta_5, q6: theta_6})
# print('R4_G_check =')
# pprint(R4_G_check)

T0_G_check = T0_G.evalf(6, subs = {q1: theta_1, q2: theta_2, q3: theta_3, q4: theta_4, q5: theta_5, q6: theta_6})
print('T0_G_check =')
pprint(T0_G_check)


print('theta_1 = %f (%f)' % (theta_1.evalf(6), joint_1))
print('theta_2 = %f (%f)' % (theta_2.evalf(6), joint_2))
print('theta_3 = %f (%f)' % (theta_3.evalf(6), joint_3))
print('theta_4 = %f (%f)' % (theta_4.evalf(6), joint_4))
print('theta_5 = %f (%f)' % (theta_5.evalf(6), joint_5))
print('theta_6 = %f (%f)' % (theta_6.evalf(6), joint_6))



O1 =
⎡0.555859   0.10163   0.825041   2.31956 ⎤
⎢                                        ⎥
⎢0.103023  -0.993269  0.0529424  0.223691⎥
⎢                                        ⎥
⎢0.824868  0.0555699  -0.562588  1.13984 ⎥
⎢                                        ⎥
⎣   0          0          0        1.0   ⎦
T0_G_check =
⎡0.555859   0.10163   0.825041   2.31956 ⎤
⎢                                        ⎥
⎢0.103023  -0.993269  0.0529424  0.223691⎥
⎢                                        ⎥
⎢0.824868  0.0555699  -0.562588  1.13984 ⎥
⎢                                        ⎥
⎣   0          0          0        1.0   ⎦
theta_1 = 0.100000 (0.100000)
theta_2 = 0.300000 (0.300000)
theta_3 = 0.100000 (0.100000)
theta_4 = -0.150000 (-0.150000)
theta_5 = 0.200000 (0.200000)
theta_6 = 0.100000 (0.100000)


## Test roll/pitch/yaw transforms

In [6]:
# Test roll/pitch/yaw transforms

# Get roll/pitch/yaw angles
gamma, beta, alpha = get_rpy(O)

print('alpha =')
pprint(alpha)
print('beta =')
pprint(beta)
print('gamma =')
pprint(gamma)

# Reconstruct rotation matrix from roll/pitch/yaw

Rxyz = get_rot_mat(gamma, beta, alpha)
print('Rxyz =')
pprint(Rxyz)

gamma, beta, alpha = get_rpy(Rxyz)
print('alpha =')
pprint(alpha)
print('beta =')
pprint(beta)
print('gamma =')
pprint(gamma)


alpha =
0.0640815205443931
beta =
0.570320412403107
gamma =
-0.0672665728170638
Rxyz =
⎡0.840000365123869   -0.100108322260879   0.533270766502233  0⎤
⎢                                                             ⎥
⎢0.0539023030757687   0.993366656214091   0.10157375667545   0⎥
⎢                                                             ⎥
⎢-0.539901776543562  -0.0565774702169301  0.839824423048626  0⎥
⎢                                                             ⎥
⎣        0                    0                   0          1⎦
alpha =
0.0640815205443931
beta =
0.547776420716252
gamma =
-0.0672665728170638
